In [13]:

from optimex import converter, optimizer
# Sets
processes = ["P1", "P2"]
system_time = list(range(2020, 2030))
process_time = [0, 1, 2, 3]
functional_flow = "F1"
intermediate_flows = ["I1", "I2"]
elementary_flows = ["CO2", "CH4"]

# Demand (known)
demand = {
    2020: 0, 2021: 0, 2022: 10, 2023: 5, 2024: 10,
    2025: 5, 2026: 10, 2027: 5, 2028: 10, 2029: 5
}

# Fixed parameters (sparse form)
foreground_production = {
    ("P1", 1): 1, ("P1", 2): 1,
    ("P2", 1): 1, ("P2", 2): 1,
}

foreground_technosphere = {
    ("P1", "I1", 0): 27.5,
    ("P2", "I2", 0): 1, 
}

foreground_biosphere = {

}

background_inventory = {
    ("db_2020", "I1", "CO2"): 1, ("db_2020", "I2", "CH4"): 1,
    ("db_2030", "I1", "CO2"): 0.9, ("db_2030", "I2", "CH4"): 0.9,
}

# Mapping: The proportion of each database used per year
mapping = {}
for y in system_time:
    mapping[("db_2020", y)] = 1 - (y - 2020) * 0.1
    mapping[("db_2030", y)] = 1 - mapping[("db_2020", y)]

# Characterization factor (impact per unit emission)
characterization = {
    "CO2": {
        2020: 8.856378067710995e-14,
        2021: 8.78632948322376e-14,
        2022: 8.716115201983699e-14,
        2023: 8.645732530491629e-14,
        2024: 8.575178705349817e-14,
        2025: 8.50445089133486e-14,
        2026: 8.433546179417147e-14,
        2027: 8.362461584725384e-14,
        2028: 8.291194044454685e-14,
        2029: 8.219740415716608e-14,
    }, 
    "CH4": {
        2020: 2.3651673669270527e-12,
        2021: 2.3651198384711042e-12,
        2022: 2.3650681065838066e-12,
        2023: 2.36501179951239e-12,
        2024: 2.3649505126261558e-12,
        2025: 2.3648838055087402e-12,
        2026: 2.364811198793221e-12,
        2027: 2.3647321707173162e-12,
        2028: 2.3646461533739266e-12,
        2029: 2.364552528630073e-12,
    },
}
scale_factor = 1e13 # Scale factor for characterization factors
characterization = {
    gas: {year: value * scale_factor for year, value in years.items()}
    for gas, years in characterization.items()
}

process_operation_start = {
    "P1": 1,
    "P2": 1,
}

process_operation_end = {
    "P1": 2,
    "P2": 2,
}


In [14]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Decision variables: how many units of each process to start at a given system time
model.x = pyo.Var(processes, system_time, domain=pyo.NonNegativeReals)

# Constraint: meet demand each year
def demand_constraint_rule(model, y):
    expr = 0
    active = False  # Flag to see if any vars are included
    for p in processes:
        for pt in process_time:
            st = y - pt
            if st in system_time and (p, pt) in foreground_production:
                active = True
                expr += model.x[p, st] * foreground_production[p, pt]
    if not active:
        return pyo.Constraint.Feasible  # no variables involved → skip
    return expr >= demand[y]



model.demand_constraint = pyo.Constraint(system_time, rule=demand_constraint_rule)

# Objective: minimize total environmental impact
def impact_expression(model):
    impact = 0
    for p in processes:
        for st in system_time:
            for pt in process_time:
                y = st + pt
                if y not in system_time:
                    continue

                # Background emissions from intermediates
                for i in intermediate_flows:
                    amount = foreground_technosphere.get((p, i, pt), 0)
                    for e in elementary_flows:
                        for db in ["db_2020", "db_2030"]:
                            mix = mapping.get((db, y), 0)
                            bg_emiss = background_inventory.get((db, i, e), 0)
                            cf = characterization[e][y]
                            impact += model.x[p, st] * amount * bg_emiss * mix * cf

                # Foreground direct emissions
                for e in elementary_flows:
                    fg_emiss = foreground_biosphere.get((p, e, pt), 0)
                    cf = characterization[e][y]
                    impact += model.x[p, st] * fg_emiss * cf

    return impact

model.objective = pyo.Objective(rule=impact_expression, sense=pyo.minimize)

# Solve
solver = pyo.SolverFactory("gurobi")
solver.options["ScaleFlag"] = 1 
# solver.options["NumericFocus"] = 3
# solver.options['Method'] = 2
# solver.options['BarHomogeneous'] = 1
results = solver.solve(model)

# Output
for p in processes:
    for y in system_time:
        val = pyo.value(model.x[p, y])
        if val > 0.001:
            print(f"{p} starts in {y}: {val:.2f} units")

print(f"Total environmental impact: {pyo.value(model.objective):.5e}")



P1 starts in 2025: 10.00 units
P1 starts in 2027: 10.00 units
P2 starts in 2021: 10.00 units
P2 starts in 2023: 10.00 units
Total environmental impact: 8.99602e+02


In [15]:
# Deployment results from the optimization, 
deployment = {
    2021: ("P2", 20), 
    2023: ("P2", 20),
    2025: ("P1", 20),
    2027: ("P1", 20)
}

def calculate_environmental_impact(deployment, foreground_technosphere, background_inventory, mapping, characterization, system_time):
    """
    Generalized method to calculate environmental impact, considering both CO2 and CH4 emissions.
    
    Args:
        deployment (dict): Dictionary containing deployment years mapped to the process ('P1' or 'P2') and corresponding units.
        foreground_technosphere (dict): Dictionary containing foreground technosphere values for each process and flow.
        background_inventory (dict): Dictionary containing background emissions for each flow and year.
        mapping (dict): Dictionary mapping years to the proportion of each database used.
        characterization (dict): Dictionary containing characterization factors for each emission type (e.g., CO2, CH4) for each year.
        system_time (list): List of years for the optimization period.
        
    Returns:
        float: Total environmental impact.
    """
    total_impact = 0

    # Loop through each year and deployment (process, units)
    for year, (process, units) in deployment.items():
        # Iterate over the inputs for each process in the foreground technosphere
        for (process_key, input_key, year_key), amount_flow in foreground_technosphere.items():
            if process_key == process:
                for gas in ["CO2", "CH4"]:  # We can handle both gases here
                    # For the selected process and input, compute the environmental impact
                    for y in system_time:
                        if year == y:
                            # Get background emissions for CO2 and CH4 for the given year
                            bg_emissions_2020 = background_inventory.get(("db_2020", input_key, gas), 0)
                            bg_emissions_2030 = background_inventory.get(("db_2030", input_key, gas), 0)

                            # Compute the emissions factor using the mapping and characterization
                            impact_2020 = bg_emissions_2020 * mapping.get(("db_2020", y), 0) * characterization[gas].get(y, 0)
                            impact_2030 = bg_emissions_2030 * mapping.get(("db_2030", y), 0) * characterization[gas].get(y, 0)

                            # Accumulate the total impact for this process and input over the years
                            total_impact += units * amount_flow * (impact_2020 + impact_2030)

    return total_impact

impact = calculate_environmental_impact(deployment, foreground_technosphere, background_inventory, mapping, characterization, system_time)

# Print the result
print(f"Total impact: {impact:.5e} units.")

Total impact: 1.79920e+03 units.


In [1]:
from optimex import converter, optimizer

import logging

# Clear existing handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Reconfigure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("This should now be visible in your notebook.")

inputs = {
        "PROCESS": ["P1", "P2"],
        "FUNCTIONAL_FLOW": ["F1"],
        "INTERMEDIATE_FLOW": ["I1", "I2"],
        "ELEMENTARY_FLOW": ["CO2", "CH4"],
        "BACKGROUND_ID": ["db_2020", "db_2030"],
        "PROCESS_TIME": [0, 1, 2, 3],
        "SYSTEM_TIME": [2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029],
        "process_operation_time": {
            "P1": (1, 2),
            "P2": (1, 2),
        },
        "demand": {
            ("F1", 2022): 10,
            ("F1", 2023): 5,
            ("F1", 2024): 10,
            ("F1", 2025): 5,
            ("F1", 2026): 10,
            ("F1", 2027): 5,
            ("F1", 2028): 10,
            ("F1", 2029): 5,
        },
        "foreground_technosphere": {
            ("P1", "I1", 0): 27.5,
            ("P2", "I2", 0): 1,
        },
        "foreground_biosphere": {
            ("P1", "CO2", 1): 0.1,
            ("P1", "CO2", 2): 0.1,
            ("P2", "CO2", 1): 0.1,
            ("P2", "CO2", 2): 0.1,
        },
        "foreground_production": {
            ("P1", "F1", 1): 1,
            ("P1", "F1", 2): 1,
            ("P2", "F1", 1): 1,
            ("P2", "F1", 2): 1,
        },
        "background_inventory": {
            ("db_2020", "I1", "CO2"): 1,
            ("db_2020", "I2", "CH4"): 1,
            ("db_2030", "I1", "CO2"): 0.9,
            ("db_2030", "I2", "CH4"): 0.9,
        },
        "mapping": {
            ("db_2020", 2020): 1.0,
            ("db_2020", 2021): 0.9,
            ("db_2020", 2022): 0.8,
            ("db_2020", 2023): 0.7,
            ("db_2020", 2024): 0.6,
            ("db_2020", 2025): 0.5,
            ("db_2020", 2026): 0.4,
            ("db_2020", 2027): 0.3,
            ("db_2020", 2028): 0.2,
            ("db_2020", 2029): 0.1,
            ("db_2030", 2020): 0,
            ("db_2030", 2021): 0.1,
            ("db_2030", 2022): 0.2,
            ("db_2030", 2023): 0.3,
            ("db_2030", 2024): 0.4,
            ("db_2030", 2025): 0.5,
            ("db_2030", 2026): 0.6,
            ("db_2030", 2027): 0.7,
            ("db_2030", 2028): 0.8,
            ("db_2030", 2029): 0.9,
        },
        # cumulative radiative forcing for timehorizon from 2020 till 2120
        "characterization": {
            key: val * 1e13
            for key, val in {  # scaling for numerical stability
                ("CO2", 2020): 8.856378067710995e-14,
                ("CO2", 2021): 8.78632948322376e-14,
                ("CO2", 2022): 8.716115201983699e-14,
                ("CO2", 2023): 8.645732530491629e-14,
                ("CO2", 2024): 8.575178705349817e-14,
                ("CO2", 2025): 8.50445089133486e-14,
                ("CO2", 2026): 8.433546179417147e-14,
                ("CO2", 2027): 8.362461584725384e-14,
                ("CO2", 2028): 8.291194044454685e-14,
                ("CO2", 2029): 8.219740415716608e-14,
                ("CH4", 2020): 2.3651673669270527e-12,
                ("CH4", 2021): 2.3651198384711042e-12,
                ("CH4", 2022): 2.3650681065838066e-12,
                ("CH4", 2023): 2.36501179951239e-12,
                ("CH4", 2024): 2.3649505126261558e-12,
                ("CH4", 2025): 2.3648838055087402e-12,
                ("CH4", 2026): 2.364811198793221e-12,
                ("CH4", 2027): 2.3647321707173162e-12,
                ("CH4", 2028): 2.3646461533739266e-12,
                ("CH4", 2029): 2.364552528630073e-12,
            }.items()
        },
    }

model_inputs = converter.ModelInputs(**inputs)
model = optimizer.create_model(model_inputs, name="test_model", flexible_operation=True)
optimizer.solve_model(model, compute_iis=True)

2025-04-29 17:33:59,073 - INFO - This should now be visible in your notebook.
2025-04-29 17:33:59,077 - INFO - Creating sets
2025-04-29 17:33:59,080 - INFO - Creating parameters
2025-04-29 17:33:59,082 - INFO - Creating variables
2025-04-29 17:33:59,086 - INFO - Creating objective function


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-10
Read LP format model from file C:\Users\HP\AppData\Local\Temp\tmp9gke9eh_.pyomo.lp
Reading time = 0.00 seconds
x1: 43 rows, 41 columns, 60 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8365U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43 rows, 41 columns and 60 nonzeros
Model fingerprint: 0x9a87802b
Model has 9 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [8e-02, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [5e+00, 1e+01]
Presolve removed 43 rows and 7 columns
Presolve time: 0.00s

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Model is infeasible


2025-04-29 17:33:59,437 - WARNING - Loading a SolverResults object with a warning status into model.name="test_model";
  - termination condition: infeasible
  - message from solver: Model was proven to be infeasible.
2025-04-29 17:33:59,437 - INFO - Solver status: infeasible
2025-04-29 17:33:59,456 - INFO - Failed to compute IIS: The Pyomo persistent interface to gurobi could not be found.


(<pyomo.core.base.PyomoModel.ConcreteModel at 0x2b78814b890>,
 {'Problem': [{'Name': 'x1', 'Lower bound': '-inf', 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 52, 'Number of variables': 41, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 41, 'Number of nonzeros': 60, 'Sense': 'minimize'}], 'Solver': [{'Status': 'warning', 'Return code': '0', 'Message': 'Model was proven to be infeasible.', 'Termination condition': 'infeasible', 'Termination message': 'Model was proven to be infeasible.', 'Wall time': '0.002000093460083008', 'Error rc': 0, 'Time': 0.3265514373779297}]})